first of we start with preprocessing the raw data

In [25]:
import pandas as pd
import numpy as np
import scipy

In [26]:
df = pd.read_csv('../data/RAW_interactions.csv')
df.drop('review', axis=1, inplace=True)
df.drop('date', axis=1, inplace=True)
df.reset_index()
df.drop_duplicates(subset=['user_id', 'recipe_id'])

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
2,8937,44394,4
3,126440,85009,5
4,57222,85009,5
...,...,...,...
1132362,116593,72730,0
1132363,583662,386618,5
1132364,157126,78003,5
1132365,53932,78003,4


In [27]:
#Set all ratings to 1 (even negative interactions are seen as interactions)
# df.loc[:,'rating'] = 1

In [28]:
#randomly drop a subset of data as we dont have enough resource to run the entire dataset
# unique_recipes = df.recipe_id.unique()
# subset = np.random.choice(unique_recipes, size=int(len(unique_recipes) / 10), replace=False, p=None)
# # Keep only the recipes that were in the randomly sampled df
# df = df[df['recipe_id'].isin(subset)]
# df.reset_index()

In [29]:
#Preprocessing step where we remove all recipes that only have a single review from a person that only has a single review
#This is done as these items will never be connected to other items and thus will never be recommended
#
# df['counts'] = df.groupby(['recipe_id'])['recipe_id'].transform('size')
# df['counts_user'] = df.groupby(['user_id'])['user_id'].transform('size')
# # 1121916 interaction in df after the and drop
# # df = df.drop(df[(df['counts'] == 1) & (df['counts_user'] == 1)].index)
#
# # 884607 interactions after the or drop
# df = df.drop(df[(df['counts'] == 1) | (df['counts_user'] == 1)].index)
# df.drop(['counts', 'counts_user'], axis=1, inplace=True)
# df.reset_index(drop=True, inplace=True)

In [30]:
df = df.drop_duplicates()
g1 = df.groupby('recipe_id', as_index=False)['user_id'].size()
g1 = g1.rename({'size': 'count_item'}, axis='columns')
g2 = df.groupby('user_id', as_index=False)['recipe_id'].size()
g2 = g2.rename({'size': 'count_user'}, axis='columns')
df = pd.merge(df, g1, how='left', on=['recipe_id'])
df = pd.merge(df, g2, how='left', on=['user_id'])
df = df[df['count_item'] >= 15]
df = df[df['count_user'] >= 15]

The recipes and users don't go from 0 to amount so if we were to put this in a matrix we would get empty columns and rows. This is not that handy so we reindex both the user_id and recipe_ids

This is a step we must not forget when entering the data in the model, as we also need to remap our input data using the same remapping that was used here

In [31]:
userSet = set(df['user_id'].to_list())
user_transform_dict = dict(map(reversed, enumerate(userSet)))
recipeSet = set(df['recipe_id'].to_list())
recipe_transform_dict = dict(map(reversed, enumerate(recipeSet)))

In [32]:
keep_nan = [k for k, v in user_transform_dict.items() if pd.isnull(v)]
tochange = df['user_id']
df['user_id'] = tochange.map(user_transform_dict).fillna(tochange.mask(tochange.isin(keep_nan)))

keep_nan = [k for k, v in recipe_transform_dict.items() if pd.isnull(v)]
tochange = df['recipe_id']
df['recipe_id'] = tochange.map(recipe_transform_dict).fillna(tochange.mask(tochange.isin(keep_nan)))


Creation of the folds for k-fold validation

In [33]:
unique_users = df.user_id.unique()

unique_users.sort(axis=0)

In [34]:
from sklearn.model_selection import train_test_split, KFold

k = 10
kf = KFold(n_splits=k, shuffle=True)
kf.get_n_splits(df)
folds = list()
for train_index, test_index in kf.split(df):
    X_train = df.iloc[train_index]
    X_test = df.iloc[test_index]
    folds.append((X_train, X_test))

Here starts the actual programming of the model

In [35]:
class popularity:
    def __init__(self):
        pass
    def train(self,data):
        data=data.sort_values('count_user',ascending=False)
        self.pop=data[data.columns[1]].to_numpy()
    def predict(self):
        return self.pop

In [36]:
class Easer:
    def __init__(self):
        pass

    def train(self, X_train, lambda_=0.5):
        self.X = X_train

        #X_train_t=X_train.copy().transpose()
        G = X_train.T.dot(X_train)  # sparse
        G = G.toarray()
        #G= X_train.T.dot(X_train).toarray() # dense
        diagIndices = np.diag_indices(G.shape[0])
        G[diagIndices] += lambda_
        P = scipy.linalg.inv(G)  # sparse
        #P = np.linalg.inv(G) # dense
        # dense diagonal retrieval (for some reason i cant get this to work on sparse matrixes something with the dimensions being wrong)
        div = -np.diag(P)
        self.B = P / div
        self.B[diagIndices] = 0

        self.pred = self.X * self.B

    def predicts(self, xu):
        #TODO give back the predicted vector given a user interaction vector
        return xu * self.B


In [37]:
import time
import pickle
from scipy import sparse

for f_idx, i in enumerate(folds):
    start = time.time()
    #for fold in folds: # removed for loop for testing
    train_data = i[0]
    ratings = train_data.rating
    idx = (train_data.user_id, train_data.recipe_id)
    print(len(df.user_id.unique()))
    print(len(df.recipe_id.unique()))
    #Here we have the user item matrix
    X_train = sparse.csc_matrix((ratings, idx), shape=(len(df.user_id.unique()), len(df.recipe_id.unique())),
                                dtype=float)
    #train model
    model_pop=popularity()
    model_pop.train(train_data)
    model = Easer()
    model.train(X_train)
    print("done")
    end = time.time()
    print("training took : ", end - start, "s")
    datafile = open("D:\\results_aiproject\\data_fold" + str(f_idx) + ".pkl", mode='wb')
    pickle.dump(i, datafile)
    modelfile = open("D:\\results_aiproject\\model_fold" + str(f_idx) + ".pkl", mode='wb')
    modelpopfile = open("D:\\results_aiproject\\model_pop_fold" + str(f_idx) + ".pkl", mode='wb')
    pickle.dump(model, modelfile)
    pickle.dump(model_pop, modelpopfile)
    datafile.close()
    modelfile.close()
    modelpopfile.close()
    break

8653
12078
done
training took :  31.029282569885254 s


In [38]:
data = pickle.load(open("D:\\results_aiproject\\data_fold0.pkl", mode='rb'))
model = pickle.load(open("D:\\results_aiproject\\model_fold0.pkl", mode='rb'))

# test_data=data[1]
# temp_result=remove_single_interaction(test_data)
# test_data.reset_index(drop=True, inplace=True)
# removed_test_data=temp_result[0]
# removed_recipes=temp_result[1]
# ratings = removed_test_data.rating
# idx = (removed_test_data.user_id, removed_test_data.recipe_id)
# x_test=sparse.csc_matrix((ratings, idx), shape=(len(test_data.user_id.unique()), len(df.recipe_id.unique())), dtype=float)
# y_pred = model.predicts(x_test)
# print(y_pred)
test_data = data[1]
predict_data = data[0]
ratings = predict_data.rating
idx = (predict_data.user_id, predict_data.recipe_id)
x_test = sparse.csc_matrix((ratings, idx), shape=(len(df.user_id.unique()), len(df.recipe_id.unique())), dtype=float)
y_pred = model.predicts(x_test)
print(y_pred)

[[ 1.29692853e-05  1.68655999e-04 -6.49266221e-03 ...  3.51492863e-03
  -8.03840436e-04 -7.51342849e-03]
 [-2.36305050e-05 -1.48826344e-05 -4.35269038e-04 ... -1.02803897e-03
  -1.93835177e-04 -1.54964534e-03]
 [ 2.17502798e-04  9.49109850e-04 -1.66775783e-03 ...  5.38801763e-03
  -2.91978342e-03  1.28300729e-02]
 ...
 [ 7.21354574e-05 -1.10500834e-03  5.29655483e-03 ... -2.47202061e-03
   2.44627106e-03 -5.98272322e-04]
 [-2.29864448e-04  1.00715644e-03 -5.73450218e-03 ... -8.12282016e-05
  -7.61354699e-04  6.23342607e-04]
 [ 4.54769058e-05  7.48103033e-04 -1.38832232e-03 ...  1.25124384e-03
  -2.61584784e-05  6.97334873e-05]]


In [39]:
#Evaluate recall@k
#Do elementwise multiplication of top K predicts and true interactions
K = 20
total = 0
correct = 0
wrong = 0
for idx, interaction in test_data.iterrows():
    user = interaction['user_id']
    user_data = predict_data.loc[(predict_data['user_id'] == user)]
    already_interacted_recipes = user_data[user_data.columns[1]].to_numpy()
    predicted = y_pred[user]
    np.put(predicted, already_interacted_recipes, -5)
    ind = (-predicted).argsort()[:K]
    recipe = interaction['recipe_id']
    if (recipe in ind):
        correct += 1
    else:
        wrong += 1
    total += 1
print("recall@%s = %s" % (str(K), str(correct / total)))

recall@20 = 0.020594965675057208


In [40]:
#recall score for popularity
data = pickle.load(open("D:\\results_aiproject\\data_fold0.pkl", mode='rb'))
model = pickle.load(open("D:\\results_aiproject\\model_pop_fold0.pkl", mode='rb'))
test_data = data[1]
predict_data = data[0]
pop=model.predict()
K = 20
total = 0
correct = 0
wrong = 0
for idx, interaction in test_data.iterrows():
    user = interaction['user_id']
    user_data = predict_data.loc[(predict_data['user_id'] == user)]
    already_interacted_recipes = user_data[user_data.columns[1]].to_numpy()
    newpop=pop[:50]
    newpop=newpop[~np.in1d(newpop,already_interacted_recipes)]
    newpop=newpop[:K]
    recipe = interaction['recipe_id']
    if (recipe in newpop):
        correct += 1
    else:
        wrong += 1
    total += 1
print("recall@%s = %s" % (str(K), str(correct / total)))

recall@20 = 0.00131242428321443
